## _*Using Qiskit Aqua for the vertex-cover problems*_

A vertex cover of a graph is a set of vertices such that each edge of the graph is incident to at least one vertex of the set. The goal of NPC problem is to minimize the size of the vertex cover. 

We will go through three examples to show:
1. How to run the optimization using the declarative approach
2. How to run the optimization using the programmatic approach
3. How how to run the optimization with the VQE.

#### The problem and the brute-force method

The problem is as follows. the graph is in the adjacent matrix form.

In [1]:
import numpy as np
from qiskit import BasicAer
from qiskit.aqua import run_algorithm
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.translators.ising import vertex_cover
from qiskit.aqua.algorithms import ExactEigensolver

np.random.seed(100)
num_nodes = 3
w = vertex_cover.random_graph(num_nodes, edge_prob=0.8, weight_range=10)
print(w)

[[0. 4. 5.]
 [4. 0. 3.]
 [5. 3. 0.]]


The brute-force method is as follows. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a vertex is either 0 (meaning the vertex is not in the cover) or 1 (meaning the vertex is in the cover). We print the binary assignment that satisfies the definition of the vertex cover and corresponds to the minimial size. 

In [2]:
def brute_force():
    # brute-force way
    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

    L = num_nodes
    max = 2**L
    minimal_v = np.inf
    for i in range(max):
        cur = bitfield(i, L)

        cur_v = vertex_cover.check_full_edge_coverage(np.array(cur), w)
        if cur_v:
            nonzerocount = np.count_nonzero(cur)
            if nonzerocount < minimal_v:
                minimal_v = nonzerocount

    return minimal_v

size = brute_force()
print('Size of the vertex cover', size)

Size of the vertex cover 2


#### Part I: Run the optimization using the declarative approach

Here the steps are:
* Create the qubit operator i.e. Ising Hamiltonian, using `vertex_cover` ising translator
* Create an EnergyInput object and a dictionary describing the algorithm and the components for Aqua to solve the problem
* Run the algorithm and get the result
* Use the result with the `vertex_cover` object to determine a solution

In [3]:
qubit_op, offset = vertex_cover.get_vertex_cover_qubitops(w)

algo_input = EnergyInput(qubit_op)
params = {
    'problem': {'name': 'ising'},
    'algorithm': {'name': 'ExactEigensolver'}
}
result = run_algorithm(params, algo_input)

x = vertex_cover.sample_most_likely(len(w), result['eigvecs'][0])
sol = vertex_cover.get_graph_solution(x)
np.testing.assert_array_equal(sol, [0, 1, 1])
print('Size of the vertex cover', np.count_nonzero(sol))

Size of the vertex cover 2


#### Part II: Run the optimization using the programmatic approach

The main difference here is running the Aqua algorithm. Here we directly construct the algorithm and then run() it to get the result. The post computation on the result is identical.

In [4]:
algo = ExactEigensolver(qubit_op)
result = algo.run()

x = vertex_cover.sample_most_likely(len(w), result['eigvecs'][0])
sol = vertex_cover.get_graph_solution(x)
np.testing.assert_array_equal(sol, [0, 1, 1])
print('Size of the vertex cover', np.count_nonzero(sol))

Size of the vertex cover 2


#### Part III: Run the optimization with the VQE

##### Declarative

We reuse the EnergyInput object we created above. VQE algorithm needs an optimizer and a variational form. Then also we need a quantum backend on which the algorithm will run.

In [5]:
params = {
    'problem': {'name': 'ising', 'random_seed': 100},
    'algorithm': {'name': 'VQE', 'operator_mode': 'paulis'},
    'optimizer': {'name': 'SPSA', 'max_trials': 200},
    'variational_form': {'name': 'RYRZ', 'depth': 3}
}
backend = BasicAer.get_backend('qasm_simulator')
result = run_algorithm(params, algo_input, backend=backend)

x = vertex_cover.sample_most_likely(len(w), result['eigvecs'][0])
sol = vertex_cover.get_graph_solution(x)
print('Size of the vertex cover', np.count_nonzero(sol))

Size of the vertex cover 2


##### Programmatic

We can create the objects directly ourselves too and run VQE for the result

In [6]:
from qiskit.aqua import aqua_globals
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RYRZ

aqua_globals.random_seed = 100

optimizer = SPSA(max_trials=200)
var_form = RYRZ(qubit_op.num_qubits, depth=3)
vqe = VQE(qubit_op, var_form, optimizer, operator_mode='paulis')

backend = BasicAer.get_backend('qasm_simulator')
result = vqe.run(backend)

x = vertex_cover.sample_most_likely(len(w), result['eigvecs'][0])
sol = vertex_cover.get_graph_solution(x)
print('Size of the vertex cover', np.count_nonzero(sol))

Size of the vertex cover 2
